Copyright (c) Alliance for Sustainable Energy, LLC and Skye Analytics, Inc. See also https://github.com/NREL/ATB-calc/blob/main/LICENSE

# Run ATB LCOE pipeline
This Jupyter notebook is used to run the ATB data processing pipeline and extract data from the ATB electricity data workbook. 
Prior to running this notebook, the ATB workbooks need to be updated. This is done in three steps:

1. Adjust the tax incentive levels from the raw values to reflect the variety of IRA incentive levels:
    1. No IRA: No ITC or PTC for any technologies
    2. Base IRA: 6% ITC or \$5/MWh PTC
    3. IRA with wage requirements: 30% ITC or \$26/MWh PTC
    4. IRA with one bonus: 40% ITC or \$29/MWh PTC
    5. IRA with both bonuses: 50% ITC or \$32/MWh PTC
3. After updating tax incentive levels, run the [Update Debt Fractions](http://localhost:8888/notebooks/Documents%2FATB-calc%2FUpdate%20Debt%20Fractions.ipynb) Jupyter notebook to calculate the expected debt to equity ratio based on the incentive levels.
4. Copy the new debt fractions into the ATB workbooks from the workbooks created by the "Update Debt Fractions" notebook

Once the previous steps are completed, save the ATB workbooks and return to this script to extract the values.

_(NOTE: I'm sure this is terrible code. Forgive me, for I know not what I do.)_

In [1]:
import os
import sys
import pandas as pd
from datetime import datetime as dt

sys.path.insert(0, os.path.dirname(os.getcwd()))
from lcoe_calculator.process_all import ProcessAll
from lcoe_calculator.tech_processors import (ALL_TECHS,
    FixedOffShoreWindProc, FloatingOffShoreWindProc, LandBasedWindProc, DistributedWindProc,
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,
    PumpedStorageHydroOneResProc,
    CoalProc, NaturalGasProc, NuclearProc, BiopowerProc,
    UtilityBatteryProc, CommBatteryProc, ResBatteryProc,
    CoalRetrofitProc, NaturalGasRetrofitProc, NaturalGasFuelCellProc)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Load the ATB data workbook & prepare for processing
Update the path for `atb_electricity_workbook` to point to your copy of the ATB electricity data
workbook downloaded from the [ATB website](https://atb.nrel.gov). 

The pipeline can be run for one, several, or all ATB electricity technologies. 


In [23]:
# The below line MUST be updated to reflect the location of the ATB workbook on your computer
filePath = "C:/Users/noah.verbeek/Documents/ATB-calc/ATB Files/"

noIRA = filePath + '1_noIRA_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
baseIRA_ITC = filePath + '2_BaseIRA_ITC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
baseIRA_PTC = filePath + '2_BaseIRA_PTC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
wageIRA_ITC = filePath + '3_IRAWage_ITC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
wageIRA_PTC = filePath + '3_IRAWage_PTC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
DCIRA_ITC = filePath + '4_IRADC_ITC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
DCIRA_PTC = filePath + '4_IRADC_PTC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
maxIRA_ITC = filePath + '5_IRAMax_ITC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx' 
maxIRA_PTC = filePath + '5_IRAMax_PTC_2024 v2 Annual Technology Baseline Workbook Errata 7-19-2024.xlsx'

# Process a couple technologies
techs = [LandBasedWindProc, UtilityPvProc, FloatingOffShoreWindProc, FixedOffShoreWindProc]

# Initiate the processor with the workbook location and desired technologies
proc_noIRA = ProcessAll(noIRA, techs)
proc_baseIRA_ITC = ProcessAll(baseIRA_ITC, techs)
proc_baseIRA_PTC = ProcessAll(baseIRA_PTC, techs)
proc_wageIRA_ITC = ProcessAll(wageIRA_ITC, techs)
proc_wageIRA_PTC = ProcessAll(wageIRA_PTC, techs)
proc_DCIRA_ITC = ProcessAll(DCIRA_ITC, techs)
proc_DCIRA_PTC = ProcessAll(DCIRA_PTC, techs)
proc_maxIRA_ITC = ProcessAll(maxIRA_ITC, techs)
proc_maxIRA_PTC = ProcessAll(maxIRA_PTC, techs)

## Run the pipeline
Now that the processor knows where the data workbook is and which technologies we are interested in, we
can kick it off. Depending on the number of requested technologies, this can take a couple minutes.
Note that calculated LCOE and CAPEX is automatically compared to the values in the workbook. Not all
technologies have LCOE and CAPEX.

In [25]:
proc_noIRA.process()
proc_baseIRA_ITC.process()
proc_baseIRA_PTC.process()
proc_wageIRA_ITC.process()
proc_wageIRA_PTC.process()
proc_DCIRA_ITC.process()
proc_DCIRA_PTC.process()
proc_maxIRA_ITC.process()
proc_maxIRA_PTC.process()

##### Processing LandbasedWind (1/4) #####
Loading data from Land-Based Wind, for Market and 20
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from workbook
Calculated LCOE matches LCOE from workbook
Loading data from Land-Based Wind, for R&D and 20
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from workbook
Calculated LCOE matches LCOE from workbook
Loading data from Land-Based Wind, for Market and 30
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from workbook
Calculated LCOE matches LCOE from workbook
Loading data from Land-Based Wind, for R&D and 30
	Loading metrics
	Loading assumptions
	Loading WACC data
	Done loading data
Calculated CAPEX matches CAPEX from workbook
Calculated LCOE matches LCOE from workbook
##### Processing UtilityPV (2/4) #####
Loading data from Solar - Utility PV, for Market and 20


## Data extraction
Congratulations! The pipeline has run and the data is available. Several options for extracting the code are shown below. 

In [34]:
flat_noIRA = proc_noIRA.data_flattened
flat_baseIRA_ITC = proc_baseIRA_ITC.data_flattened
flat_baseIRA_PTC = proc_baseIRA_PTC.data_flattened
flat_wageIRA_ITC = proc_wageIRA_ITC.data_flattened
flat_wageIRA_PTC = proc_wageIRA_PTC.data_flattened
flat_DCIRA_ITC = proc_DCIRA_ITC.data_flattened
flat_DCIRA_PTC = proc_DCIRA_PTC.data_flattened
flat_maxIRA_ITC = proc_maxIRA_ITC.data_flattened
flat_maxIRA_PTC = proc_maxIRA_PTC.data_flattened

flat_noIRA["Tax Credit Amount"] = "No IRA"
flat_baseIRA_ITC["Tax Credit Amount"] = "Base IRA"
flat_baseIRA_PTC["Tax Credit Amount"] = "Base IRA"
flat_wageIRA_ITC["Tax Credit Amount"] = "IRA Wage"
flat_wageIRA_PTC["Tax Credit Amount"] = "IRA Wage"
flat_DCIRA_ITC["Tax Credit Amount"] = "IRA DC Bonus"
flat_DCIRA_PTC["Tax Credit Amount"] = "IRA DC Bonus"
flat_maxIRA_ITC["Tax Credit Amount"] = "Max IRA"
flat_maxIRA_PTC["Tax Credit Amount"] = "Max IRA"

flat_noIRA["Tax Credit"] = "None"
flat_baseIRA_ITC["Tax Credit"] = "ITC"
flat_baseIRA_PTC["Tax Credit"] = "PTC"
flat_wageIRA_ITC["Tax Credit"] = "ITC"
flat_wageIRA_PTC["Tax Credit"] = "PTC"
flat_DCIRA_ITC["Tax Credit"] = "ITC"
flat_DCIRA_PTC["Tax Credit"] = "PTC"
flat_maxIRA_ITC["Tax Credit"] = "ITC"
flat_maxIRA_PTC["Tax Credit"] = "PTC"

flatData = pd.concat([flat_noIRA, 
                      flat_baseIRA_ITC, flat_baseIRA_PTC, 
                      flat_wageIRA_ITC, flat_wageIRA_PTC,
                      flat_DCIRA_ITC,   flat_DCIRA_PTC,
                      flat_maxIRA_ITC,  flat_maxIRA_PTC], axis=0)

flatData

,Parameter,Case,TaxCreditCase,CRPYears,Technology,DisplayName,Scenario,variable,value,Tax Credit Amount,Tax Credit
0,Inflation Rate,Market,None,20,LandbasedWind,*,*,2022,0.02739,No IRA,None
1,Interest Rate Nominal,Market,None,20,LandbasedWind,*,Advanced,2022,0.07,No IRA,None
2,Interest Rate Nominal,Market,None,20,LandbasedWind,*,Moderate,2022,0.07,No IRA,None
3,Interest Rate Nominal,Market,None,20,LandbasedWind,*,Conservative,2022,0.07,No IRA,None
4,Calculated Interest Rate Real,Market,None,20,LandbasedWind,*,Advanced,2022,0.041474,No IRA,None
...,...,...,...,...,...,...,...,...,...,...,...
108571,CAPEX,R&D,None,30,OffShoreWind,Offshore Wind - Class 6,Moderate,2050,3407.835663,Max IRA,PTC
108572,CAPEX,R&D,None,30,OffShoreWind,Offshore Wind - Class 6,Conservative,2050,4139.032616,Max IRA,PTC
108573,CAPEX,R&D,None,30,OffShoreWind,Offshore Wind - Class 7,Advanced,2050,2987.686734,Max IRA,PTC
108574,CAPEX,R&D,None,30,OffShoreWind,Offshore Wind - Class 7,Moderate,2050,3603.496308,Max IRA,PTC


In [36]:
# Save flattened data to as a CSV
flatData.to_csv('atb_data_flat.csv', index=False)

## Exploring the data in the notebook
The extracted and calculated parameters can also be viewed directly in the notebook if desired. Note that the data is a combination of all technologies that were processed and may need to be filtered. Working with a single technology processor as described in the `Tech Processor.ipynb` notebook may be easier depending on your goals. Data is internally stored as a [pandas](https://pandas.pydata.org) data frame.